In [1]:
#import stokespy
#from stokespy import get_HMI_data, get_SP_data, StokesParamCube, StokesParamMap, StokesProfile
import datetime
import astropy
import astropy.units as u
from astropy.coordinates import SkyCoord, SpectralCoord
from astropy.wcs.wcsapi import SlicedLowLevelWCS, HighLevelWCSWrapper

import sunpy.map
from sunpy.net import Fido, attrs

import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib
#import ndcube

matplotlib.use('TkAgg')

# Setup the autoreload function.
%load_ext autoreload
%autoreload 2

In [2]:
user_dir = os.getcwd() + '/Data/Hinode/'
#user_date = '20170822_171104'   # AR near limb
user_date = '20170905_030404'  # AR near center

### Generate the scan file list.
level1_dir = user_dir + 'Level1/' + user_date
level1_files = []
for file in sorted(os.listdir(level1_dir)):
    if not file.endswith(".fits"):
        continue
    level1_files.append(os.path.join(level1_dir, file))

In [3]:
# Load one file for reference
SP_level1 = astropy.io.fits.open(level1_files[0])

### Read all the Level 1 scan data 
Nx = len(level1_files)
Nstokes, Ny, Nwav = SP_level1[0].data.shape
level1_data = np.zeros((Nx, Nstokes, Ny, Nwav))

# Because of discrepancies between the lvl1 and lvl2 headers
# it is useful for debuggin purposes to keep track of the individual
# scan file headers. The approximate location of the central pixel
# can be calculated as the median
head_all = []
xcen_a = []
ycen_a = []
for ix, file in enumerate(level1_files):
    SP_lvl1_obj = astropy.io.fits.open(file)
    level1_data[ix] = SP_lvl1_obj[0].data
    head_all.append(SP_lvl1_obj['Primary'].header)
    xcen_a.append(head_all[ix]['XCEN'])
    ycen_a.append(head_all[ix]['YCEN'])

level1_data = level1_data.transpose(1, 3, 2, 0) # data axes order: stokes, wav, y, x

In [4]:
### Read the Level 2 fit data. 
level2_fname = user_dir + '/Level2/' + user_date + '.fits'
SP_level2 = astropy.io.fits.open(level2_fname)

Ny, Nx = SP_level2['Field_Strength'].data.shape
level2_data = np.zeros((3, Ny, Nx))
level2_data[0] = SP_level2['Field_Strength'].data
level2_data[1] = SP_level2['Field_Inclination'].data
level2_data[2] = SP_level2['Field_Azimuth'].data
level2_data.shape

### Build WCS objects.
head1 = SP_level1['PRIMARY'].header
head2 = SP_level2['PRIMARY'].header

In [5]:
### Read the Level 2.1 fit data. 
level21_fname = user_dir + '/Level2.1/' + user_date + '_L2.1.fits'
SP_level21 = astropy.io.fits.open(level21_fname)

#Ny, Nx = SP_level2['Field_Strength'].data.shape
#level2_data = np.zeros((3, Ny, Nx))
#level2_data[0] = SP_level2['Field_Strength'].data
#level2_data[1] = SP_level2['Field_Inclination'].data
#level2_data[2] = SP_level2['Field_Azimuth'].data
#level2_data.shape

### Build WCS objects.
#head1 = SP_level1['PRIMARY'].header
head21 = SP_level21['PRIMARY'].header

FileNotFoundError: [Errno 2] No such file or directory: '/home/gabriel/Desktop/Science/StokesPY/stokespy_notebooks/Data/Hinode//Level2.1/20170822_171104_L2.1.fits'

In [108]:
head21

SIMPLE  =                    T / Written by IDL:  Fri Nov 27 00:08:02 2020      
BITPIX  =                    8 /                                                
NAXIS   =                    0 /                                                
NAXIS1  =                  485 /                                                
NAXIS2  =                  512 /                                                
EXTEND  =                    T / File contains extensions                       
XCEN    =              96.3010 /                                                
YCEN    =              40.5247 /                                                
XSCALE  =             0.297140 /                                                
YSCALE  =             0.319978 /                                                
TSTART  = '2017-09-05T03:04:04.495' /                                           
TEND    = '2017-09-05T03:34:53.245' /                                           
MAPID   = '124491  '        

In [6]:
SP_level2[0].header

SIMPLE  =                    T / Written by IDL:  Sat Nov 28 05:53:05 2020      
BITPIX  =                    8 /                                                
NAXIS   =                    0 /                                                
NAXIS1  =                  511 /                                                
NAXIS2  =                  512 /                                                
EXTEND  =                    T / File contains extensions                       
XCEN    =             -427.830 /                                                
YCEN    =             -3.79296 /                                                
XSCALE  =             0.297140 /                                                
YSCALE  =             0.319978 /                                                
TSTART  = '2017-08-22T17:11:04.097' /                                           
TEND    = '2017-08-22T17:43:23.003' /                                           
MAPID   = '124396  '        

In [17]:
SP_level2[0].header['YSCALE']

0.319978

In [18]:
head1

SIMPLE  =                    T /image conforms to FITS standard                 
BITPIX  =                   16 /bits per data value                             
NAXIS   =                    3 /number of axes                                  
NAXIS1  =                  112 /                                                
NAXIS2  =                  512 /                                                
NAXIS3  =                    4 /                                                
EXTEND  =                    T /file may contain extensions                     
DATE    = '2017-09-14T16:17:55.000' /                                           
DATE_RF0= '2017-09-14T16:17:55.000' /                                           
TELESCOP= 'HINODE  '           /                                                
INSTRUME= 'SOT/SP  '           /                                                
MDP_CLK =            916097019 /                                                
ORIGIN  = 'JAXA/ISAS, SIRIUS

In [5]:
x_arr = SP_level2[38].data
SP_level2[38].header
y_arr = SP_level2[39].data

In [8]:
x_arr.shape, y_arr.shape

((512, 511), (512, 511))

In [9]:
SP_level2[41].header

XTENSION= 'BINTABLE'           / Written by IDL:  Sat Nov 28 05:53:07 2020      
BITPIX  =                    8 /                                                
NAXIS   =                    2 / Binary table                                   
NAXIS1  =                 2044 / Number of bytes per row                        
NAXIS2  =                    1 / Number of rows                                 
PCOUNT  =                    0 / Random parameter count                         
GCOUNT  =                    1 / Group count                                    
TFIELDS =                    1 / Number of columns                              
EXTNAME = 'Mechanical_Slit_Position' / Merlin : Mechanical Slit Position        
TFORM1  = '511E    '           / Real*4 (floating point)                        
TTYPE1  = 'SLITPOS '           / Label for column 1                             

In [10]:
%matplotlib widget

plt.imshow(x_arr, origin='lower')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
plt.plot(x_arr[200,:])

In [64]:
np.median(y_arr[200,:])

22.67912

In [30]:
xxx = SP_level2[41].data

In [36]:
xxx[0][0].shape

(485,)

In [38]:
%matplotlib widget

plt.plot(xxx[0][0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# Look at the centeral position for each header.
nslit = len(head_all)
xcen_a = np.zeros(nslit)
ycen_a = np.zeros(nslit)

for i in range(nslit):
    xcen_a[i] = head_all[i]['XCEN']
    ycen_a[i] = head_all[i]['YCEN']

In [13]:
y_arr.shape

(512, 511)

In [13]:
# Calculate the center based on MERLIN

#mx = (max(l1info.xpos) - min(l1info.xpos))/2.
#fxaddpar, header, 'XCEN', min(l1info.xpos)+mx
#fxaddpar, header, 'YCEN', mean(l1info.ypos)
# This ends up being the average between the two extrema.

mx = (xcen_a.max() + xcen_a.min())/2

In [14]:
mx

96.30099639574999

In [7]:
%matplotlib widget

plt.plot(xcen_a, label='XCEN$_{lvl1}$')
plt.plot(ycen_a, label='YCEN$_{lvl1}$')
plt.plot(x_arr[200,:], '--', label='XCEN$_{lvl2}$ (Ext 38)')
plt.plot(np.median(y_arr, axis=0), '--', label='YCEN$_{lvl2}$ (Ext 39)')

plt.xlabel('Fiber number')
plt.ylabel('Center location (arcsec)')

plt.title('20170905_030404')

plt.plot(len(xcen_a)/2, np.median(xcen_a), 'xb', label='Median(XCEN$_{lvl1}$)')
plt.plot(len(ycen_a)/2, np.median(ycen_a), 'ob', label='Median(YCEN$_{lvl1}$)')

plt.plot(len(xcen_a)/2, SP_level2[0].header['Xcen'], 'xm', label='Head$_{lvl2}$[XCEN]')
plt.plot(len(ycen_a)/2, SP_level2[0].header['Ycen'], 'om', label='Head$_{lvl2}$[YCEN]')

print(np.median(xcen_a), np.median(ycen_a))
print(np.median(x_arr), np.median(y_arr))
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

129.963 40.598
129.963 40.35192


In [10]:
%matplotlib widget

plt.imshow(x_arr, origin='lower')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
y_arr.shape

(512, 485)